In [2]:
import pyprind
import pandas as pd
import os
import nltk
from nltk.corpus import stopwords
import time
t1 = time.time()

In [5]:
basepath ='/Users/gupta/Desktop/ML-DTU-Workshop/aclImdb'
labels = {'pos': 1, 'neg': 0}
pbar = pyprind.ProgBar(50000)

df = pd.DataFrame()
for s in ('test', 'train'):
    for l in ('pos', 'neg'):
        path = os.path.join(basepath, s, l)
        for file in os.listdir(path):
            with open(os.path.join(path, file), 'r', encoding='utf-8') as infile:
                txt = infile.read()
            df = df.append([[txt, labels[l]]], ignore_index=True)
            pbar.update()
df.columns = ['review', 'sentiment']
#To shuffle the data
import numpy as np
np.random.seed(0)
df = df.reindex(np.random.permutation(df.index))
df.to_csv('movie_data.csv',index = False)

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:18:39


In [7]:
df = pd.DataFrame()
df = pd.read_csv('movie_data.csv')
print(df.head(10))
print(df.loc[0,'review'][-50:])


                                              review  sentiment
0  In 1974, the teenager Martha Moxley (Maggie Gr...          1
1  OK... so... I really like Kris Kristofferson a...          0
2  ***SPOILER*** Do not read this, if you think a...          0
3  hi for all the people who have seen this wonde...          1
4  I recently bought the DVD, forgetting just how...          0
5  Leave it to Braik to put on a good show. Final...          1
6  Nathan Detroit (Frank Sinatra) is the manager ...          1
7  To understand "Crash Course" in the right cont...          1
8  I've been impressed with Chavez's stance again...          1
9  This movie is directed by Renny Harlin the fin...          1
is seven.<br /><br />Title (Brazil): Not Available


In [9]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
#UNIGRAM MODEL
count = CountVectorizer(ngram_range=(2,2))
docs = np.array([
        'The sun is shining',
        'The weather is sweet',
        'The sun is shining and the weather is sweet'])
bag = count.fit_transform(docs)
print(count.vocabulary_)
print(bag.toarray())
from sklearn.feature_extraction.text import TfidfTransformer
tfidf = TfidfTransformer()
np.set_printoptions(precision=2)

print(tfidf.fit_transform(bag).toarray())


{'the sun': 5, 'sun is': 4, 'is shining': 1, 'the weather': 6, 'weather is': 7, 'is sweet': 2, 'shining and': 3, 'and the': 0}
[[0 1 0 0 1 1 0 0]
 [0 0 1 0 0 0 1 1]
 [1 1 1 1 1 1 1 1]]
[[0.   0.58 0.   0.   0.58 0.58 0.   0.  ]
 [0.   0.   0.58 0.   0.   0.   0.58 0.58]
 [0.43 0.33 0.33 0.43 0.33 0.33 0.33 0.33]]


In [17]:
import re

def preprocessor(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)    
    text = re.sub('[\W]+',' ',text.lower())+ ' '.join(emoticons).replace('-', '')
    return text
df['review'] = df['review'].apply(preprocessor)
print(preprocessor('is seven also<br\>known as :-( (Available)'))

from nltk.stem.porter import PorterStemmer
porter = PorterStemmer()

def tokenizer(text):
    return text.split()
def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]
from sklearn.feature_extraction.text import TfidfVectorizer
nltk.download('stopwords')
stop = stopwords.words('english')
print(list(w for w in tokenizer_porter('a runner likes running and runs alot') if w not in stop))


is seven alsoknown as available :(
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\gupta\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
['runner', 'like', 'run', 'run', 'alot']


In [ ]:
X_train = df.loc[:25000,'review'].values
y_train = df.loc[:25000,'sentiment'].values
X_test = df.loc[25000:,'review'].values
y_test = df.loc[25000:,'sentiment'].values

from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV
#Convert raw documents into the sparse matrix
tfidf = TfidfVectorizer(strip_accents=None,lowercase=False,preprocessor=None,)


lr_tfidf = Pipeline([('vect', tfidf),
                ('clf', LogisticRegression(random_state=0))])


param_grid = [{'vect__ngram_range': [(1, 1)],
            'vect__stop_words': [stop, None],
            'vect__tokenizer': [tokenizer, tokenizer_porter],
            'clf__penalty': ['l1', 'l2'],
            'clf__C': [1.0, 10.0, 100.0] },
            {'vect__ngram_range': [(1, 1)],
            'vect__stop_words': [stop, None],
            'vect__tokenizer': [tokenizer, tokenizer_porter],
            'vect__use_idf':[False],
            'vect__norm':[None],
            'clf__penalty': ['l1', 'l2'],
            'clf__C': [1.0, 10.0, 100.0]},
            ]

gs_lr_tfidf = GridSearchCV(lr_tfidf, param_grid,scoring='accuracy',cv=5,verbose=1,n_jobs=1)

gs_lr_tfidf.fit(X_train, y_train)


Fitting 5 folds for each of 48 candidates, totalling 240 fits


In [ ]:
print('Best parameter set:', gs_lr_tfidf.best_params_)
print('CV Accuracy:',gs_lr_tfidf.best_score_)


clf = gs_lr_tfidf.best_estimator_
print('Test Accuracy:',clf.score(X_test, y_test))

t2 = time.time()
print("total time spent in hyperrparamter training ",(t2-t1)/60,"mins")
